In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Posledny dataset, forma hracov z posledneho zapasu. Pocet golov v poslednych zapasoch pre kazdy tim.

In [2]:
gameInput = pd.read_csv("./../../data/raw/game.csv")
goalieInput = pd.read_csv("./../../data/raw/game_goalie_stats.csv")
skaterStatsInput = pd.read_csv("./../../data/raw/game_skater_stats.csv")

In [3]:
goalies = goalieInput.loc[:,['game_id','team_id','shots','saves']].groupby(['game_id','team_id']).sum().reset_index()

In [4]:
goalies['percentage'] = (goalies['saves'] / goalies['shots'])

In [5]:
goalies

,game_id,team_id,shots,saves,percentage
0,2010020001,8,24,21,0.875000
1,2010020001,10,28,26,0.928571
2,2010020002,4,31,29,0.935484
3,2010020002,5,27,24,0.888889
4,2010020003,12,29,26,0.896552
...,...,...,...,...,...
22863,2018030415,19,39,38,0.974359
22864,2018030416,6,29,28,0.965517
22865,2018030416,19,31,27,0.870968
22866,2018030417,6,20,16,0.800000


In [6]:
gameInput = gameInput.sort_values(by=['game_id'])

In [7]:
gameInput

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
11078,2010020001,20102011,R,2010-10-07,2010-10-07T23:00:00Z,8,10,2,3,home win REG,right,Air Canada Centre,/api/v1/venues/null,America/Toronto,-4,EDT
10725,2010020002,20102011,R,2010-10-07,2010-10-07T23:00:00Z,4,5,3,2,away win REG,left,CONSOL Energy Center,/api/v1/venues/null,America/New_York,-4,EDT
9625,2010020003,20102011,R,2010-10-07,2010-10-07T16:00:00Z,12,30,4,3,away win REG,right,Hartwall Areena,/api/v1/venues/null,America/Chicago,-5,CDT
10256,2010020004,20102011,R,2010-10-08,2010-10-08T02:00:00Z,16,21,3,4,home win OT,right,Pepsi Center,/api/v1/venues/null,America/Denver,-6,MDT
10673,2010020005,20102011,R,2010-10-08,2010-10-08T02:00:00Z,20,22,0,4,home win REG,left,Rexall Place,/api/v1/venues/null,America/Edmonton,-6,MDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,P,2019-06-02,2019-06-02T00:00:00Z,6,19,7,2,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11430,2018030414,20182019,P,2019-06-04,2019-06-04T00:00:00Z,6,19,2,4,home win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11431,2018030415,20182019,P,2019-06-07,2019-06-07T00:00:00Z,19,6,2,1,away win REG,left,TD Garden,/api/v1/venues/5085,America/New_York,-4,EDT
11432,2018030416,20182019,P,2019-06-10,2019-06-10T00:00:00Z,6,19,5,1,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT


Teraz si musíme vytvoriť dva ďalšie stĺpce, ktoré budú obsahovať, počet víťazstiev domáceho alebo hosťujúceho tímu v predchádzajúcich 5 zápasoch.

In [8]:
dataSet1 = pd.DataFrame(columns=['game_id','season', 'date_time', 'game1home','game1away', 'game1isHome','game2home', \
                                 'game2away','game2isHome','game3home','game3away','game3isHome','game4home','game4away',\
                                 'game4isHome','game5home','game5away','game5isHome','game6home','game6away', 'game6isHome','game7home', \
                                 'game7away','game7isHome','game8home','game8away','game8isHome','game9home','game9away',\
                                 'game9isHome','game10home','game10away','game10isHome', \
                                 'home_team_players', 'away_team_players',\
                                 'home_team_id', 'away_team_id', \
                                 'total'])

In [9]:
def getPlayers(game_id, team_id, percentage):
#     print(game_id)
    games = skaterStatsInput.loc[((skaterStatsInput.game_id == game_id) & (skaterStatsInput.team_id == team_id))]\
        .sort_values(by=['timeOnIce']).head(15)
    array = np.array([percentage])
    i = 0
    for index, row in games.iterrows():
        value = row.goals * 0.7 + row.assists * 0.5 + row.shots * 0.3
        array = np.append(array, value)
        i += 1
    if(i<15):
        for j in range(i, 16):
            array = np.append(array, 0)
    return array

In [11]:
# games = pd.DataFrame(columns=['game_id', 'season', 'time', 'home_scored', 'home_conceded', 'home_win','home_percentage', 'home_id', 'away_scored', 'away_conceded', 'away_win','away_percentage', 'away_id'])

for index, row in gameInput.iterrows():
    d = {}
    homeS = 0;
    homeC = 0;
    homeW = 0;
    awayS = 0;
    awayC = 0;
    awayW = 0;
    homePerc = 50;
    awayPerc = 50;
    check = 0;
    for i in range(1,11):
        d['game' + str(i) + 'home'] = 0
        d['game' + str(i) + 'away'] = 0
        if(i>5):
            d['game' + str(i) + 'isHome'] = 0
        else:
            d['game' + str(i) + 'isHome'] = 1
    if not (gameInput.loc[((gameInput['home_team_id'] == row['home_team_id']) & (gameInput['date_time_GMT'] < row['date_time_GMT']))].tail(5).empty):
        homeLast = gameInput.loc[((gameInput['home_team_id'] == row['home_team_id']) & (gameInput['date_time_GMT'] < row['date_time_GMT']))].tail(5)
        i = 1
        for index1, row1 in homeLast.iterrows():
            
            d['game' + str(i) + 'home'] = row1['home_goals']
            d['game' + str(i) + 'away'] = row1['away_goals']
            if not goalies.loc[(goalies['game_id'] == row1['game_id']) & (goalies['team_id'] == row['home_team_id'])].empty:
                homePerc1 = goalies.loc[(goalies['game_id'] == row1['game_id']) & (goalies['team_id'] == row['home_team_id'])]
                homePerc = homePerc1['percentage'].values[0]
            i += 1
        if(i > 5):
            check += 1
        d['home_team_players'] = getPlayers(homeLast.tail(1).game_id.values[0], row['home_team_id'], homePerc)
    if not (gameInput.loc[((gameInput['away_team_id'] == row['away_team_id']) & (gameInput['date_time_GMT'] < row['date_time_GMT']))].tail(5).empty):
        awayLast = gameInput.loc[((gameInput['away_team_id'] == row['away_team_id']) & (gameInput['date_time_GMT'] < row['date_time_GMT']))].tail(5)
        i = 6
        for index1, row1 in awayLast.iterrows():
            d['game' + str(i) + 'home'] = row1['home_goals']
            d['game' + str(i) + 'away'] = row1['away_goals']
            if not goalies.loc[(goalies['game_id'] == row1['game_id']) & (goalies['team_id'] == row['away_team_id'])].empty:
                awayPerc1 = goalies.loc[(goalies['game_id'] == row1['game_id']) & (goalies['team_id'] == row['away_team_id'])]
                awayPerc = awayPerc1['percentage'].values[0]
            i += 1
        if(i > 5):
            check += 1
        d['away_team_players'] = getPlayers(awayLast.tail(1).game_id.values[0], row['away_team_id'], awayPerc)
    d['game_id'] = row['game_id'];
    d['date_time'] = row['date_time_GMT'];
    d['season'] = row['season'];
    d['home_team_id'] = row['home_team_id'];
    d['away_team_id'] = row['away_team_id'];
    d['total'] = row['away_goals'] + row['home_goals'];
    if(check == 2):
        dataSet1.at[index, :] = d
        print(row['game_id'])

2010020107
2010020118
2010020127
2010020131
2010020133
2010020134
2010020136
2010020139
2010020141
2010020143
2010020144
2010020145
2010020146
2010020148
2010020150
2010020151
2010020154
2010020157
2010020158
2010020159
2010020160
2010020161
2010020162
2010020164
2010020165
2010020168
2010020169
2010020170
2010020171
2010020173
2010020175
2010020176
2010020179
2010020180
2010020181
2010020183
2010020184
2010020185
2010020186
2010020187
2010020189
2010020190
2010020193
2010020194
2010020195
2010020196
2010020197
2010020198
2010020199
2010020200
2010020201
2010020202
2010020203
2010020204
2010020205
2010020206
2010020207
2010020208
2010020209
2010020210
2010020211
2010020212
2010020213
2010020214
2010020215
2010020216
2010020217
2010020218
2010020219
2010020220
2010020221
2010020222
2010020223
2010020224
2010020225
2010020226
2010020227
2010020228
2010020229
2010020230
2010020231
2010020232
2010020233
2010020234
2010020235
2010020236
2010020237
2010020238
2010020239
2010020240
2010020241

In [13]:
dataSet1

,game_id,season,date_time,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,...,game9away,game9isHome,game10home,game10away,game10isHome,home_team_players,away_team_players,home_team_id,away_team_id,total
9770,2010020107,20102011,2010-10-24T00:30:00Z,2,3,1,2,3,1,4,...,0,0,0,0,0,"[0.972972972972973, 1.4, 0.0, 0.0, 1.9, 0.5, 0...","[0.9130434782608695, 1.6, 1.0, 0.3, 1.1, 0.3, ...",16,29,5
9116,2010020118,20102011,2010-10-26T23:30:00Z,4,2,1,2,3,1,1,...,6,0,0,0,0,"[0.8571428571428571, 0.0, 0.0, 0.3, 0.89999999...","[0.9629629629629629, 0.0, 1.5, 1.3, 1.6, 1.4, ...",4,7,9
10304,2010020127,20102011,2010-10-28T00:30:00Z,2,3,1,4,3,1,3,...,6,0,2,3,0,"[0.925, 0.3, 0.3, 0.6, 0.6, 0.3, 0.3, 0.3, 0.0...","[0.9090909090909091, 0.0, 0.0, 0.8999999999999...",16,26,4
10262,2010020131,20102011,2010-10-28T23:00:00Z,2,3,1,2,5,1,3,...,0,0,0,0,0,"[0.9655172413793104, 0.0, 0.3, 0.3, 0.0, 0.6, ...","[0.8974358974358975, 0.0, 0.0, 0.0, 1.59999999...",29,22,5
11068,2010020133,20102011,2010-10-29T00:00:00Z,4,1,1,4,3,1,2,...,0,0,0,0,0,"[0.8947368421052632, 0.3, 0.8, 0.0, 0.8, 1.5, ...","[0.9090909090909091, 0.0, 0.0, 0.0, 0.6, 1.2, ...",18,19,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,2019-06-02T00:00:00Z,1,2,1,2,1,1,4,...,2,0,0,4,0,"[0.9615384615384616, 0.5, 0.8999999999999999, ...","[1.0, 0.3, 0.0, 0.3, 0.3, 0.3, 0.0, 0.8, 2.3, ...",19,6,9
11430,2018030414,20182019,2019-06-04T00:00:00Z,2,1,1,4,5,1,2,...,4,0,2,7,0,"[0.7391304347826086, 0.8, 0.0, 0.0, 1.1, 0.0, ...","[0.9310344827586207, 0.3, 1.5, 2.2, 0.0, 0.6, ...",19,6,6
11431,2018030415,20182019,2019-06-07T00:00:00Z,4,3,1,5,2,1,6,...,2,0,2,3,0,"[0.918918918918919, 0.3, 0.0, 0.0, 0.8, 0.3, 1...","[0.9130434782608695, 0.0, 0.8, 1.0, 0.0, 0.3, ...",6,19,3
11432,2018030416,20182019,2019-06-10T00:00:00Z,4,5,1,2,1,1,5,...,7,0,4,2,0,"[0.9130434782608695, 0.0, 0.0, 0.6, 1.7, 0.0, ...","[0.918918918918919, 1.1, 0.3, 0.3, 0.3, 0.6, 0...",19,6,6


In [67]:
array = np.array([])
for col in dataSet1.loc[0]:
    array = np.append(array, col)
array

array(['2011030221.0', '20112012.0', '2012-04-29T19:00:00Z', '3', '5',
       '1', '2', '1', '1', '8', '4', '1', '3', '10', '1', '5', '1', '1',
       '3', '5', '0', '2', '1', '0', '8', '4', '0', '3', '10', '0', '5',
       '1', '0', '0.8846153846153846', '0.0', '0.0', '0.0', '0.0', '0.3',
       '1.5999999999999999', '1.6', '1.0', '1.2', '0.0', '1.0', '0.3',
       '0.3', '0.3', '1.0', '0.8888888888888888', '1.0', '0.5', '0.0',
       '0.3', '0.8', '1.1', '2.3', '1.1', '1.6', '0.0', '1.5', '0.0',
       '1.5', '0.3', '1.7999999999999998', '4', '1', '7'], dtype='<U32')

In [63]:
dataSet1.loc[0].values

array([2011030221, 20112012, '2012-04-29T19:00:00Z', 3, 5, 1, 2, 1, 1, 8,
       4, 1, 3, 10, 1, 5, 1, 1, 1, 2, 0, 2, 3, 0, 4, 2, 0, 3, 0, 0, 2, 3,
       0,
       array([0.96774194, 0.        , 0.        , 0.        , 0.        ,
       0.3       , 1.6       , 1.6       , 1.        , 1.2       ,
       0.        , 1.        , 0.3       , 0.3       , 0.3       ,
       1.        ]),
       array([0.95555556, 1.        , 0.5       , 0.        , 0.3       ,
       0.8       , 1.1       , 2.3       , 1.1       , 1.6       ,
       0.        , 1.5       , 0.        , 1.5       , 0.3       ,
       1.8       ]),
       4, 1, 7], dtype=object)

In [64]:
dataSet2 = dataSet1.sample(frac=1).reset_index(drop=True)

In [65]:
dataSet2

,game_id,season,date_time,game1home,game1away,game1isHome,game2home,game2away,game2isHome,game3home,...,game9away,game9isHome,game10home,game10away,game10isHome,home_team_players,away_team_players,home_team_id,away_team_id,total
0,2013021178,20132014,2014-04-08T00:00:00Z,2,4,1,7,2,1,5,...,3,0,3,2,0,"[0.8095238095238095, 0.0, 0.0, 0.6, 1.0, 0.8, ...","[0.9230769230769231, 0.0, 0.3, 0.0, 0.0, 0.0, ...",52,30,1
1,2017020914,20172018,2018-02-19T21:00:00Z,1,2,1,2,4,1,4,...,5,0,6,1,0,"[0.8, 0.0, 0.0, 0.0, 0.8999999999999999, 0.899...","[0.7391304347826086, 0.3, 2.2, 0.6, 0.0, 1.2, ...",20,6,3
2,2014020402,20142015,2014-12-07T20:00:00Z,3,1,1,3,4,1,2,...,4,0,4,5,0,"[0.9090909090909091, 0.3, 1.0, 0.0, 1.1, 0.3, ...","[0.84, 0.0, 0.0, 1.0, 2.7, 0.8, 0.5, 1.5999999...",52,24,7
3,2010020925,20102011,2011-02-26T21:00:00Z,5,2,1,0,2,1,2,...,3,0,4,0,0,"[0.9259259259259259, 0.0, 0.6, 0.3, 0.6, 0.0, ...","[0.8787878787878788, 0.3, 0.0, 0.3, 0.3, 0.899...",26,21,7
4,2012020338,20122013,2013-03-07T03:00:00Z,2,1,1,3,2,1,4,...,4,0,2,4,0,"[0.9393939393939394, 0.6, 0.3, 1.2999999999999...","[0.9354838709677419, 0.5, 0.5, 0.3, 2.0, 0.899...",24,27,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11261,2015020215,20152016,2015-11-10T03:00:00Z,0,3,1,4,1,1,4,...,4,0,6,0,0,"[0.9444444444444444, 0.0, 0.6, 0.0, 0.3, 1.599...","[0.7777777777777778, 0.3, 0.3, 0.0, 0.3, 0.3, ...",24,53,7
11262,2011020712,20112012,2012-01-22T03:30:00Z,1,0,1,0,1,1,5,...,2,0,6,1,0,"[0.96, 0.3, 0.3, 0.6, 0.8999999999999999, 0.0,...","[0.8181818181818182, 0.3, 1.2999999999999998, ...",26,21,4
11263,2018020551,20182019,2018-12-23T00:00:00Z,3,4,1,2,6,1,4,...,1,0,3,1,0,"[0.8857142857142857, 0.0, 0.0, 0.0, 0.3, 0.6, ...","[0.9032258064516129, 0.0, 0.8999999999999999, ...",7,24,3
11264,2012020412,20122013,2013-03-16T23:00:00Z,4,3,1,2,1,1,3,...,0,0,3,0,0,"[0.975, 0.0, 0.3, 0.3, 0.3, 0.8999999999999999...","[0.9, 0.0, 0.3, 1.2, 0.0, 0.6, 0.8999999999999...",29,27,1


In [44]:
dataTrain1 = dataSet2.loc[0:int(len(dataSet2)*0.8)]
dataTest1 = dataSet2.loc[int(len(dataSet2)*0.8) + 1:]

In [66]:
dataSet5TrainX = dataTrain1.drop('season',1).drop('game_id',1).drop('date_time',1).drop('total',1).drop('home_team_id',1).drop('away_team_id',1)
dataSet5TrainY = pd.DataFrame(dataTrain1.total)
dataSet5TestX = dataTest1.drop('season',1).drop('game_id',1).drop('date_time',1).drop('total',1).drop('home_team_id',1).drop('away_team_id',1)
dataSet5TestY = pd.DataFrame(dataTest1.total)


In [67]:
dataSet5TrainXFinal = []
for index, row in dataSet5TrainX.iterrows():
    array = np.array([])
    for col in row:
        array = np.append(array, col)
    dataSet5TrainXFinal.append(array)

dataSet5TestXFinal = []
for index, row in dataSet5TestX.iterrows():
    array = np.array([])
    for col in row:
        array = np.append(array, col)
    dataSet5TestXFinal.append(array)

In [68]:
dataSet5TestXFinal = pd.DataFrame(dataSet5TestXFinal)
dataSet5TrainXFinal = pd.DataFrame(dataSet5TrainXFinal)

In [69]:
dataSet5TrainXFinal.to_csv('.\..\..\data\processed\dataSet5TrainX.csv', encoding='utf-8', index=False)
dataSet5TrainY.to_csv('.\..\..\data\processed\dataSet5TrainY.csv', encoding='utf-8',index=False)
dataSet5TestXFinal.to_csv('.\..\..\data\processed\dataSet5TestX.csv', encoding='utf-8', index=False)
dataSet5TestY.to_csv('.\..\..\data\processed\dataSet5TestY.csv', encoding='utf-8', index=False)

In [71]:
print(len(dataTrain1.loc[dataTrain1.total == 6])/len(dataTrain1))
print(len(dataTrain1.loc[dataTrain1.total == 5])/len(dataTrain1))
print(len(dataTest1.loc[dataTest1.total == 6])/len(dataTest1))
print(len(dataTest1.loc[dataTest1.total == 5])/len(dataTest1))

0.10640186397425941
0.24187285032730502
0.09454061251664447
0.2596537949400799
